<a href="https://colab.research.google.com/github/swishswish123/tracked_surgery_simulations/blob/main/tracked_endoscope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2) TRACKED ENDOSCOPE SIMULATION

In this notebook we want to study the accuracy of a tracked endoscope, and what the expected accuracy of an overlay will be. The overlay accuracy could be measured in terms of accuracy in 3D in the endoscope camera frame, or in terms of 2D pixel accuracy.

Please first read tracked_pointer.ipynb before this notebook for a better understanding of the problem and the maths behind it.

## imports and utility functions


In [ ]:
# github repo which contains all the images
!git clone https://github.com/swishswish123/tracked_surgery_simulations.git repo


In [ ]:
from scipy.spatial.transform import Rotation as spr


In [ ]:
def extrinsic_matrix_to_vecs(matrix):
    """
    extract_rigid_body_parameters(matrix)
    extracts parameters from transformation matrix

    Args:
        matrix: 4x4 transformation matrix

    Returns:
        list of rigid body parameters [tx, ty, tz, rx, ry, rz]

    """
    t = matrix[0:3, 3]
    r = matrix[0:3, 0:3]
    rot = spr.from_matrix(r)
    euler = rot.as_euler('xyz', degrees=True)
    return [t[0], t[1], t[2], euler[0], euler[1], euler[2]]

def extrinsic_vecs_to_matrix(params):
    """
    rigid_body_parameters_to_matrix(params)
    converts a list of rigid body parameters to transformation matrix

    Args:
        params: list of rigid body parameters [tx, ty, tz, rx, ry, rz]

    Returns:
        4x4 transformation matrix of these parameters

    """
    matrix = np.eye(4)
    
    matrix[0][3] = params[0]
    matrix[1][3] = params[1]
    matrix[2][3] = params[2]

    r = (spr.from_euler('xyz', [params[3], params[4], params[5]], degrees=True)).as_matrix()
    matrix[0:3, 0:3] = r
    return matrix


## Background, System Layout, goal and assumptions

The following diagram shows the layout of the navigation system and the different components involved in the surgery when using a tracked endoscope for AR.



In [ ]:
Image(filename="./repo/assets/endoscope_setup.png")


The goal is to be able to display some segmented piece of information from the pre-operative MRI in MRI coordinates onto the endoscopic video:

In [ ]:
Image(filename="./repo/assets/endoscope_goal.png")


The goal is therefore turning some segmented MRI coordinates to Endoscopic Image (EndIm) coordinates.

X<sub>EndIm</sub> = T X<sub>MRI</sub>

where T is composed of the following transforms:

T = <sup>EndIm</sup>T<sub>EndP</sub> * 
    <sup>EndP</sup>T<sub>EndRef</sub> * 
    <sup>EndRef</sup>T<sub>Cam</sub>  * 
    <sup>Cam</sup>T<sub>PatRef</sub> * 
    <sup>PatRef</sup>T<sub>MRI</sub>
    


Here is an visual representation of the endoscope setup with all the transforms:


In [ ]:
Image(filename="endosim_demo/images/endoscope_setup_transforms.png")

In the following sections we will go step by step on how to go from each of these transforms

## Assumptions

In order to make this a like-for-like comparison to the tracked pointer simulation, we should keep most of the reference data identical. The difference now, is we swap a pointer for an endoscope. We will need an additional hand-eye calibration, and reference data to project from camera space onto image space, but we leave all other data the same. 

In [ ]:
Image(filename="endosim_demo/images/endoscope_setup_assumptions.png")


In [ ]:
# P - , the length of the endoscope.
LENGTH_OF_ENDOSCOPE = 180 # use 300 after merging

# D - z distance from camera to plane where everything is located
DISTANCE_FROM_CAM = 2000 # since the camera and patient reference are aligned in the x and y directions, only distance is in z

# 0 - angle of pointer
ENDOSCOPE_ANGLE = 45

# Yc - distances from tumour to patient reference
TUMOUR_PATREF = 300  

# NDI quotes 0.25mm for Polaris Spectra, some papers estimate it at 0.17mm
#TYPICAL_TRACKING_SIGMA = 0.25

# For Model 2 and 3, using an endoscope, this determines the distance of a target of interest from the endoscope.
#working_distance = 50

# for simulation to be reproducible
NUMBER_SAMPLES = 10

X_T = 100 # head length (about 20cm)
Y_T = 130 # menton to top of head (about 25cm)
Z_T = 80 # head bredth (about 15cm)

END_SIGMA=0.5
SIGMA_STEP=0.01

## Defining references

We will use the same reference coordinates as used by the pointer in this simulation to ensure the two are easily comparable.

The patient reference will be set up exactly the same. However, what was previously the pointer reference will now be the endoscope reference (EndRef) and so instead of the pointer length there will be an endoscope length between the endoscope's camera, and the first marker.

In [ ]:
def create_pat_ref():
    """
    Create reference coordinates of a marker pattern in a numpy matrix.

    This function creates a numpy matrix containing the reference coordinates of a marker pattern used for
    pose estimation. The marker pattern consists of four markers labeled A, B, C and D. The reference coordinates
    of each marker point are defined in a right-handed reference frame as follows:

        Marker A: (0, 0, 0)
        Marker B: (41.02, 0, 28.59)
        Marker C: (88.00, 0, 0)
        Marker D: (40.45, 0, -44.32)

    The function returns a 4x4 numpy matrix containing the homogenous coordinates of each marker point, with the
    last element of each row set to 1.

    Returns:
        numpy.ndarray: A 4x4 numpy matrix containing reference coordinates of a marker pattern. Each row represents a
        marker point in homogenous coordinates.
    """

    # Encoding the reference marker points into a numpy matrix
    pat_ref = np.zeros((4, 4))
    # marker A (0) -> (0,0,0)

    # marker B (1) -> (41.02 ,0,28.59)
    pat_ref[1][0] = 41.02  # x
    pat_ref[1][2] = 28.59  # z

    # marker C (2) -> C = (88.00 ,0, 0)
    pat_ref[2][0] = 88  # x

    # marker D (3) -> (40.45,0,-44.32)
    pat_ref[3][0] = 40.45  # x
    pat_ref[3][2] = -44.32  # z

    # adding 1 to last row to make coordinates homogenous
    pat_ref[0][3] = 1.0
    pat_ref[1][3] = 1.0
    pat_ref[2][3] = 1.0
    pat_ref[3][3] = 1.0
    return pat_ref


def create_end_ref():
    """
    Creates a numpy matrix representing the endoscope reference coordinates.

    Returns:
    end_ref (numpy matrix): A 4x4 numpy matrix containing the pointer reference coordinates
                            as row vectors in homogenous coordinates. The four rows represent
                            the markers A, B, C, and D in that order.
    """
    # Creating pointer reference (from datasheet). Using homogenous (4 numbers, x,y,z,1) as row vectors.
    end_ref = np.zeros((4, 4))

    # marker A (0) -> 0,0,0

    # marker B (1) -> 0,50,0
    end_ref[1][1] = 50  # y

    # marker c (2) -> 25,100,0
    end_ref[2][0] = 25  # x
    end_ref[2][1] = 100  # y

    # marker d (3) -> -25, 135, 0
    end_ref[3][0] = -25  # x
    end_ref[3][1] = 135  # y

    # adding 1 to 3rd dimension to turn to homogeneous coordinates
    end_ref[0][3] = 1
    end_ref[1][3] = 1
    end_ref[2][3] = 1
    end_ref[3][3] = 1

    return end_ref

## Obtaining transforms


Remember we want at the end a composition of all these transforms:

T = <sup>EndIm</sup>T<sub>EndP</sub> * 
    <sup>EndP</sup>T<sub>EndRef</sub> * 
    <sup>EndRef</sup>T<sub>Cam</sub>  * 
    <sup>Cam</sup>T<sub>PatRef</sub> * 
    <sup>PatRef</sup>T<sub>MRI</sub>
  
Most of these are obtained in a similar way to the pointer simulation although flipped as we are going from the MRI coordinates. Since we are assuming the layout and relative positions of each reference marker, we are able to obtain the transformations between each coordinate system.

The function below generates these transformations with the given pointer length and the parameters defined in the assumptions section.

In [ ]:
def get_transforms(endoscope_length=200, ENDOSCOPE_ANGLE=45,DISTANCE_FROM_CAM=2000, TUMOUR_PARTEF=40  ):
    """
    Returns a set of coordinate transformation matrices that convert points from one reference frame to another.

    Args:
        endoscope_length (float): Length of the endoscope in millimeters. Default is 200.
        ENDOSCOPE_ANGLE (float): z angle at which endoscope is angled
        DISTANCE_FROM_CAM (float): distance from camera to patient
        TUMOUR_PARTEF (float): distance from tumour to patient reference

    Returns:
        tuple: A tuple containing the following eight transformation matrices:
        - PatRef_T_MRI: A 4x4 transformation matrix that converts points from the MRI reference frame to the patient reference frame.
        - Cam_T_PatRef: A 4x4 transformation matrix that converts points from the patient reference frame to the camera reference frame.
        - EndRef_T_Cam: A 4x4 transformation matrix that converts points from the camera reference frame to the endoscope reference frame. 
        - EndP_T_EndRef: A 4x4 transformation matrix that converts points from the endoscope reference frame to the endoscope tip reference frame.

        and the following numpy arrays containing the homogenous coordinates of the points in the given reference frame
        - end_ref_marker:  endRef in the pointer reference frame.
        - end_ref_cam: endRef in the camera reference frame.
        
        - pat_ref_marker: patRef in the patient reference frame.
        - pat_ref_cam: patRef in the camera reference frame.

    """

    # 1) obtaining EndRef_T_EndP
    #print(f'endoscope length: {endoscope_length}')
    # Creating pointer reference in the pointer reference frame.
    end_ref_marker = create_end_ref() # marker coords
    # Point reference to point tip: (pointer length translation in y)
    EndP_T_EndRef = extrinsic_vecs_to_matrix(
        [0, pointer_length, 0, 0, 0, 0]) # create transform of all points depending on pointer's length
    # invert to get tip to ref:
    #EndRef_T_EndP = np.linalg.inv(EndP_T_EndRef)

    # 2) obtaining Cam_T_EndRef
    # Converting the marker points to the camera reference frame by applying a
    # rotation of ENDOSCOPE_ANGLE degrees about the z-axis followed by a
    # translation of DISTANCE_FROM_CAM along the z-axis of camera.
    rotate_about_z = extrinsic_vecs_to_matrix([0, 0, 0, 0, 0, ENDOSCOPE_ANGLE])
    translate_away_from_camera = extrinsic_vecs_to_matrix([0, 0, DISTANCE_FROM_CAM, 0, 0, 0])
    Cam_T_EndRef = translate_away_from_camera @ rotate_about_z
    end_ref_cam = multiply_points_by_transform(end_ref_marker, Cam_T_EndRef)
    EndRef_T_Cam = np.linalg.inv(Cam_T_EndRef)

    # 3) obtaining Cam_T_PatRef
    # PatRef to Cam (add dist to cam to z plus x translation to right)
    pat_ref_marker = create_pat_ref()
    # translating to correct location 
    translate_along_x = extrinsic_vecs_to_matrix([TUMOUR_PARTEF, 0, 0, 0, 0, 0])
    Cam_T_PatRef = translate_along_x @ translate_away_from_camera
    pat_ref_cam = multiply_points_by_transform(pat_ref_marker, Cam_T_PatRef)
    #PatRef_T_Cam = np.linalg.inv(Cam_T_PatRef)

    # 4) obtaining PatRef_T_MRI
    PatRef_T_MRI = extrinsic_vecs_to_matrix([X_T, Y_T, Z_T, 0, 0, 0])
    #MRI_T_PatRef = np.linalg.inv(PatRef_T_MRI)

    return EndP_T_EndRef,EndRef_T_Cam,Cam_T_PatRef,PatRef_T_MRI  , end_ref_marker, end_ref_cam, pat_ref_marker, pat_ref_cam
